# COMP4167 Natural Language Processing
# Practical I - Part I
# Tokenization, Term-Document Matrix, TF-IDF and Text classification

In this notebook you will practice some of the methods discusssed in the lectures to address an NLP task, text classification.

In this notebook you will:
- Perform the process of tokenization
- Build a Term-Document Matrix (using some methods like Counting words and TFIDF) as the feature extraction method
- Apply a machine learning classifier to predict or classify a tweet as real or fake.

## Problem Description
Emergency agencies started using social media and especially Twitter to identify quickly when disasters are hppening. In this problem,you’re tasked to build a machine learning model that predicts which Tweets are related to real disaster and which are not relevant. The dataset has 10,000 tweets that were manually classified.

This is a [Kaggle competition](https://www.kaggle.com/c/nlp-getting-started) to getting started in NLP.

## Importing the libraries

- make sure the libraries are already installed using pip or conda
- install the latest version of the libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn

%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score

## Setup the data for pre-processing 

Data are assumed to be in the 'data' folder which includes two files:
- `train.csv`: the training data set
- `test.csv`: the testing data set

In [ ]:
data_folder_name = 'data'
train_filename = 'train.csv'

train_path = data_folder_name +'/'+ train_filename 

# Relevant columns for this exercise 
TEXT_COLUMN = 'text'
TARGET_COLUMN = 'target'

### Loading the datasets

Each sample in the train and test set has the following information:

- The text of a tweet
- A keyword from that tweet (although this may be blank!)
- The location the tweet was sent from (may also be blank)

The task is to predict whether a given tweet is about a real disaster (1) or not (0).

In [ ]:
# Read the tweets of the train dataset
# we will use this data for the training by spliting it into further training and validation set
data = pd.read_csv(train_path)

In [ ]:
# Sample some data from the training set
# re-running the cell will produce another sample
data.sample(10)

In [ ]:
# Extract only the text and target columns from our dataframe, 
# for this exercise we will assume the other fields are not important
data = data[[TEXT_COLUMN, TARGET_COLUMN]]

In [ ]:
# sample 10 strings of class 1 (disaster)
list(data[data['target']==1].sample(10)['text'])

In [ ]:
# sample 10 strings of class 0 (not a disaster)
list(data[data['target']==0].sample(10)['text'])

### Splitting the dataset into train and test set

We split the train dataset into a train and validation dataset so we can evaluate the result with cross-validation. 
- we use 80-20 data split
- set the random_state which help re-producing the same results every time we run the code

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data[TEXT_COLUMN], 
                                                  data[TARGET_COLUMN].values,
                                                  test_size=0.20,
                                                  random_state=0)
# check the size of our datasets
print('Size of training set:',X_train.shape)
print('Size of validation set:',X_val.shape)

# Feature extraction

In this exercise we will extract TF/IDF features.
- We will use the implmentation of TF/IDF in scikit-learn
- In the implmentation we will use lower case 
- We will drop the words that occured less than two times in the documents
- The TF/IDF vectorizer is built on the training data and is then used to extract features from both training and validation sets 

In [ ]:
# Try multiple ways of calculating features.
#   Create the numericalizer TFIDF for lowercase
tfidf = TfidfVectorizer(decode_error='ignore', lowercase=True, min_df=2)
#   Numericalize the train dataset
train = tfidf.fit_transform(X_train.values.astype('U'))
#   Numericalize the test dataset
val = tfidf.transform(X_val.values.astype('U'))

In [ ]:
print('Train size: ',train.shape)
print('Val size: ',val.shape)

### Sample some words from the dictionary

In [ ]:
dictionary = np.asarray(tfidf.get_feature_names_out())
print(dictionary[np.random.randint(0,len(dictionary),size=20)])

## Train the models

The extracted features will be used to train classifiers in to order to predict which tweets belong to class 0 or class 1.

We will test here three classifiers: 
- Naive Bayes
- SVM
- XBGboost Classifier

### Naive Bayes

One of the most commonly used classifier for text classification: Naive Bayes using multinomial models. 
The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification).

In [ ]:
# create the model, train it on the train dataset and print the scores
model = MultinomialNB() # as implemented in sklearn
model.fit(train, y_train)
print("Train score:", model.score(train, y_train))
print("Validation score:", model.score(val, y_val))

#### Evaluate the Naive Bayes classifier
First we create some helper functions to plot the results:

In [ ]:
# Create the confusion matrix
def plot_confusion_matrix(y_test, y_pred):
    ''' Plot the confusion matrix for the target labels and predictions '''
    cm = confusion_matrix(y_test, y_pred)

    # Create a dataframe with the confusion matrix values
    df_cm = pd.DataFrame(cm, range(cm.shape[0]),
                  range(cm.shape[1]))

    # Plot the confusion matrix
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True,fmt='.0f',cmap="YlGnBu",annot_kws={"size": 10}) # font size
    plt.show()

In [ ]:
# ROC Curve
# plot no skill
# Calculate the points in the ROC curve
def plot_roc_curve(y_test, y_pred):
    ''' Plot the ROC curve for the target labels and predictions'''
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    roc_auc= auc(fpr,tpr)
    plt.figure(figsize=(12, 12))
    ax = plt.subplot(121)
    ax.set_aspect(1)
    
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
# Predicting the Test set results
y_pred = model.predict(val)

#print the classification report to highlight the accuracy with f1-score, precision and recall
print(metrics.classification_report(y_val, y_pred))
plot_confusion_matrix(y_val, y_pred)

# ROC curve needs the raw model probabilities, not the model predictions
y_pred_prob = model.predict_proba(val)[:,1]
plot_roc_curve(y_val, y_pred_prob)


### Suport Vector Machine Classifier
Alternatively, we can use SVM algorithm to predict if a tweet is fake or real, it is just a binary classifcation problem. 
For SVM it is important to optimise the parameters of the kernel used. Here we use a GridSearch to optimise the hyper paramters of an RBF Kernel. A similar process can be done for other kernels like Linear or Gaussian.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the parameters to tune
parameters = { 
    'C': [1.0, 10],
    'gamma': [1, 'auto', 'scale']
}
# Tune hyperparameters  using Grid Search and a SVM model
model = GridSearchCV(SVC(kernel='rbf', probability=True), parameters, cv=5, n_jobs=-1).fit(train, y_train)

Once our model is trained we can evaluate the performance as we did previously:

In [ ]:
# Predicting the validation set 
y_pred = model.predict(val)

print(metrics.classification_report(y_val, y_pred))
plot_confusion_matrix(y_val, y_pred)

y_pred_prob = model.predict_proba(val)[:,1]
plot_roc_curve(y_val, y_pred_prob)


This simple SVM increases the performance of our model but not significantly.

### XGBoost classifier

XGBoost is an implementation of gradient boosted decision trees designed for speed and performance. XGBoost is an advanced version of gradient boosting. Rather than training all of the models in isolation of one another, boosting trains models in succession, with each new model being trained to correct the errors made by the previous ones. Models are added sequentially until no further improvements can be made.

XGBoost provides a wrapper class to allow any models to be treated like classifiers or regressors in the scikit-learn framework. This means we can use the full scikit-learn library with XGBoost models.

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score


def f1_metric(ytrue,preds):
    ''' Return the F1 Score value for the preds and true values, ytrue '''
    return 'f1_score', f1_score((preds>=0.5).astype('int'), ytrue, average='macro'), True

# set the model parameters
params = {
    'learning_rate': 0.06,
    'n_estimators': 1500,
    'colsample_bytree': 0.5,
    'metric': 'f1_score'
}

full_clf = LGBMClassifier(**params)

# Fit or train the xgboost model
full_clf.fit(train.astype(np.float32), y_train, eval_set=[(train.astype(np.float32), y_train), (val.astype(np.float32), y_val)],
             verbose=400, eval_metric=f1_metric)

#Show the results
print("train score:", full_clf.score(train.astype(np.float32), y_train))
print("val score:", full_clf.score(val.astype(np.float32), y_val))

Now, we can predict on the test dataset to get the results and compare with others methods.

In [ ]:
# Predicting the Test set results
Y_pred = full_clf.predict(val.astype(np.float32))

print(metrics.classification_report(y_val, y_pred))
plot_confusion_matrix(y_val, y_pred)

y_pred_prob = model.predict_proba(val)[:,1]
plot_roc_curve(y_val, y_pred_prob)


## Visualize some results from our text

Another tool we can use to analyze the results is a WordCloud where we can draw the most relevant words in the fake tweets and real tweets. 

In [ ]:
# visualize the data on a WordCloud
from wordcloud import WordCloud

def visualize(label):
    words = ''
    for msg in data[data[TARGET_COLUMN] == label][TEXT_COLUMN]:
        msg = msg.lower()
        words += msg + ' '
    wordcloud = WordCloud(width=600, height=600).generate(words)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
visualize(1)
visualize(0)